<a href="https://colab.research.google.com/github/wiiiiish/2022-machine_learning/blob/main/mysql_insert_final_0511.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls

drive  sample_data


In [2]:
%cd drive/MyDrive/Colab\ Notebooks/2022-04-27\ 데이터

/content/drive/MyDrive/Colab Notebooks/2022-04-27 데이터


In [3]:
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install pymysql

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-libmysqlclient-dev is already the newest version (1.0.4).
default-libmysqlclient-dev set to manually installed.
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
     |████████████████████████████████| 43 kB 1.1 MB/s 


In [5]:
import pandas as pd
import numpy as np
import pymysql # mysql 연결
import pickle # 모델 불러올 때 사용
from datetime import date, datetime, timezone, timedelta # 시간 관련 라이브러리
import pickle

In [6]:
# 모델 불러오기
with open('LinearRegression_s_pred_model.pickle', 'rb') as f:
    s_model = pickle.load(f)

In [7]:
# 시간 맞추기
# hours=9 는 현재 한국시간
# 우리는 한 시간 후의 시간을 불러와야 하기 때문에 hours=10 으로 설정
K_KST = timezone(timedelta(hours=9))
H_KST = timezone(timedelta(hours=10))

In [21]:
# DB 연결
from pymysql.converters import convert_bit
host = "project-db-stu.ddns.net"
port = 3307
username = "wish"
password = "1234"
db = 'wish'

conn = pymysql.connect(host=host, port=port, user=username, password=password, db=db)

# select 현재 시간, 데이터 번호 (real_data의 마지막 데이터 불러오기)
sql_state1 = f"select r_seq, r_date, r_aod from s_real_data order by r_date desc limit 1"
s_real = pd.read_sql_query(sql_state1, conn)
s_r_seq = s_real['r_seq']
s_r_date = s_real['r_date']
s_r_now_aod = s_real['r_aod']

# 불러온 데이터의 시간 추출
s_now_time = f"{s_r_date[0]}"
print(f"s_now_time : {s_now_time}")


# s_now_time 의 한 시간 후의 시간
s_now_time_slice = s_now_time[:18]
s_pred_t = (int)(s_now_time_slice[11:13]) + 1
print(f"s_pred_t : {s_pred_t}")

if s_pred_t < 10:
  s_pred_t = f"0{s_pred_t}"

timeList = list(s_now_time)
timeList[11:13] = f"{s_pred_t}"
s_pred_time = ''.join(s for s in timeList)
s_pred_time_slice = s_pred_time[:18]
print(f"s_pred_time_slice : {s_pred_time_slice}")

# s_now_time 의 한 시간 전의 시간
s_before_t = (int)(s_now_time_slice[11:13]) - 1
print(f"s_before_t : {s_before_t}")

if s_before_t < 10:
  s_before_t = f"0{s_before_t}"

timeList[11:13] = f"{s_before_t}"
s_before_time = ''.join(s for s in timeList)
s_before_time_slice = s_before_time[:18]
print(f"s_before_time_slice : {s_before_time_slice}")

# select 한 시간 후의 날씨
sql_state2 = f"select * from temp_weather where date_time like '{s_pred_time_slice}%'"
s_after_weather = pd.read_sql_query(sql_state2, conn)
after_hour_weather = pd.DataFrame(s_after_weather)

# select 한 시간 전의 발전량
sql_state3 = f"select r_aod from s_real_data where r_date like '{s_before_time_slice}'"
s_before_aod = pd.read_sql_query(sql_state3, conn)
before_hour_aod = pd.DataFrame(s_before_aod)
print(f"before_hour_aod : {before_hour_aod}")

# 모델에 넣을 데이터 형식 맞추기
after_hour_weather['now_aod'] = s_r_now_aod
after_hour_weather['before_aod'] = before_hour_aod

test = after_hour_weather.loc[:, 'temperature':]
X_test = np.array(test)

pred_aod = s_model.predict(X_test)
s_pred_aod = pred_aod[0]

print(f"s_pred_aod : {s_pred_aod}")

if s_pred_aod < 0 :
  s_pred_aod = 0

# DB insert into s_predict
sql_state3 = f"insert into s_prediction values (null, {s_r_seq}, '{s_pred_time}', {s_pred_aod})"
cursor = conn.cursor()
cursor.execute(sql_state3)
conn.commit()

s_now_time : 2022-05-11 12:25:06
s_pred_t : 13
s_pred_time_slice : 2022-05-11 13:25:0
s_before_t : 11
s_before_time_slice : 2022-05-11 11:25:0
before_hour_aod : Empty DataFrame
Columns: [r_aod]
Index: []


,date_time,temperature,precipitation,wind_speed,humidity,radiation,total_cloudiness,visibility,ground_temp,now_aod
0,2022-05-11 13:25:00,30.4,0.0,1.5,63.6,0.9,0.0,2000.0,37.4,575
